In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings('ignore')
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
from tensorflow.python.keras.backend import set_session
import tensorflow as tf
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2)  
from tensorflow.keras import Input, Model, layers
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.compat.v1.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
import csv
import random
import itertools
import time
import pickle
from data_utils import *

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, Dropout, Flatten, Bidirectional
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Permute, Reshape
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:21:00.0, compute capability: 8.6



## Import dataset

In [2]:
sampling_rate = 100
window_size = 550
stride = 50
f = '/home/nesl/209as_sec/human_act/Data/Activity_Dataset/'

X_tr, Y_tr, X_test, Y_test = import_auritus_activity_dataset(dataset_folder = f, 
                                use_timestamp=False, 
                                shuffle=True, 
                                window_size = window_size, stride = stride, 
                                return_test_set = True, test_set_size = 300,channels=0)

print(X_tr.shape)
print(Y_tr.shape)
print(X_test.shape)
print(Y_test.shape)

X_tr = X_tr.swapaxes(1,2)
X_test = X_test.swapaxes(1,2)

100%|█████████████████████████████████████████| 399/399 [00:12<00:00, 32.74it/s]


(14671, 550, 6)
(14671, 9)
(300, 550, 6)
(300, 9)


## Training

In [3]:
def model_variant(model, num_feat_map, dim, network_type,p):
    print(network_type)
    if network_type == 'ConvLSTM':
        model.add(Permute((2, 1, 3))) 
        model.add(Reshape((-1,num_feat_map*dim)))
        model.add(Bidirectional(LSTM(128, return_sequences=False, stateful=False)))
    if network_type == 'CNN':
        
        model.add(Flatten())
        model.add(Dense(64, activation='relu'))
        model.add(BatchNormalization()) 
        model.add(Dropout(p))

        
def model_conv(model, num_feat_map,p,b):
    model.add(Conv2D(num_feat_map, kernel_size=(1, 10),    
                 activation='relu',
                 input_shape=(dim, win_len, 1),
                 padding='same'))
    
    model.add(Conv2D(num_feat_map, kernel_size=(1, 10), activation='relu',padding='same'))
    
    if (b==1):
        model.add(BatchNormalization()) 
    model.add(Conv2D(num_feat_map, kernel_size=(1, 10), activation='relu',padding='same'))
    
    if (b==1):
        model.add(BatchNormalization()) 
    model.add(MaxPooling2D(pool_size=(1, 3)))
    
    model.add(Conv2D(num_feat_map, kernel_size=(1, 10), activation='relu',padding='same')) 
    model.add(Conv2D(num_feat_map, kernel_size=(1, 10), activation='relu',padding='same'))
    if (b==1):
        model.add(BatchNormalization()) 
    model.add(MaxPooling2D(pool_size=(1, 2)))
    model.add(Dropout(p))
    
    model.add(Conv2D(num_feat_map, kernel_size=(1, 10), activation='relu',padding='same'))  
    if (b==1):
        model.add(BatchNormalization()) 
    model.add(MaxPooling2D(pool_size=(1, 2)))
    
    model.add(Dropout(p))
    
def model_LSTM(model,p):
    model.add(LSTM(num_hidden_lstm, 
               input_shape=(win_len,dim), 
               return_sequences=True))
    model.add(Dropout(p))
    model.add(LSTM(num_hidden_lstm, return_sequences=False))
    model.add(Dropout(p))
    
def model_output(model):
    model.add(Dense(num_classes, activation='softmax'))

In [4]:
batch_size = 64
num_feat_map = 128
num_hidden_lstm = 128
num_classes = 9


network_type = 'CNN'
_, dim, win_len = X_tr.shape

X_tr = np.reshape(X_tr, (-1, dim, win_len, 1))
X_test = np.reshape(X_test, (-1, dim, win_len, 1))

In [5]:
p=0.5 #Dropout
b = 1 #BatchNorm
model = Sequential()

if network_type=='CNN' or network_type=='ConvLSTM':
    model_conv(model, num_feat_map,p,b)
    model_variant(model, num_feat_map, dim, network_type,p)
if network_type=='LSTM':
    model_LSTM(model,p)
       
model_output(model)    
model.summary()

CNN
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 6, 550, 128)       1408      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 6, 550, 128)       163968    
_________________________________________________________________
batch_normalization (BatchNo (None, 6, 550, 128)       512       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 550, 128)       163968    
_________________________________________________________________
batch_normalization_1 (Batch (None, 6, 550, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 6, 183, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 183, 128)    

In [6]:
model_name = 'cnn.h5'
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

checkpoint = ModelCheckpoint(model_name, save_best_only=True, monitor='val_accuracy')
model.fit(X_tr,Y_tr,
              epochs=40,
              validation_split=0.1,
              callbacks=[checkpoint],
              shuffle=True,
              verbose=1)

Epoch 1/40
413/413 [==============================] - 12s 23ms/step - loss: 1.1119 - accuracy: 0.5841 - val_loss: 1.8399 - val_accuracy: 0.4087
Epoch 2/40
413/413 [==============================] - 9s 22ms/step - loss: 0.6570 - accuracy: 0.7445 - val_loss: 1.4413 - val_accuracy: 0.5163
Epoch 3/40
413/413 [==============================] - 9s 22ms/step - loss: 0.5351 - accuracy: 0.7892 - val_loss: 0.4712 - val_accuracy: 0.8202
Epoch 4/40
413/413 [==============================] - 9s 22ms/step - loss: 0.4329 - accuracy: 0.8271 - val_loss: 0.3952 - val_accuracy: 0.8590
Epoch 5/40
413/413 [==============================] - 9s 22ms/step - loss: 0.3788 - accuracy: 0.8528 - val_loss: 0.4055 - val_accuracy: 0.8413
Epoch 6/40
413/413 [==============================] - 9s 22ms/step - loss: 0.3242 - accuracy: 0.8744 - val_loss: 0.2981 - val_accuracy: 0.8726
Epoch 7/40
413/413 [==============================] - 9s 22ms/step - loss: 0.3245 - accuracy: 0.8706 - val_loss: 0.5522 - val_accuracy: 0.779

## Evaluate Model

In [7]:
new_model = load_model(model_name)
new_model.evaluate(X_test,Y_test)

10/10 [==============================] - 0s 13ms/step - loss: 0.0287 - accuracy: 0.9900


[0.02869269996881485, 0.9900000095367432]